### 웹 크롤링 나무위키 사이트 분석 및 시각화
<Step1 . 크롤링> : 크롤링으로 웹 데이터 가져오기

[웹크롤링 라이버러시 사용하기]
- beautifulsoup, requests 라이브러리로 웹 크롤러를 만듬
- request는 특정 url로 부터 html 문서를 가져오는 작업 수행
- 나무위키와 같은 페이지는 html문서가 javascript 동적 로딩되는 경우 가능
- request대신 셀레니움(selenium) 라이브러리를 이용해 크롬 브라우저로 동적 웹크롤링 수행
- selenium은 웹브라우저를 자동으로 구동해주는 라이브러리
- selenium을 사용하기 위해 크롬 드라이버를 이용해 크롬 브라우저 자동으로 구동 -> 크롬드라이버 필요

[beautifulsoup과 selenium을 이용한 웹 크롤링]
- anaconda prompt 혹은 cmd에서 패키지 설치
- pip install selenium
- pip install beautifulsoup4
 
[크롬 브라우저 업데이트 및 크롬 드라이버 설치]
- 크롬 브라우저 설정에서 최신 버전으로 업데이트
- 크롬 드라이버 사이트에서 브라우저에 맞는 드라이버 다운로드
- chromedriver.exe 파일을 노트북 파일 경로에 이동

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[ BeautifulSoup의 select() VS find_add()]
- html의 특정요소 선택
- select, select_one의 경우 CSS 선택자를 이용하는 것처럼 사용 가능
- select의 경우 후손이나 자손 요소를 CSS 처럼 선택 가능
- 예) soup.select("dl > dt > a")
- find_all, find 의 경우 하나의 태그(name="table")나 하나의 클래스(class="tables")를 선택
- find의 경우 후손이나 자손 요소를 직접 선택할 수 없어 한번 더 변수에 담든지 루프 문을 이용하애 함
- find_all(class="ah_roll"), find(name="table")

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re # 정규식 표현을 위한 모듈

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows) 지정
excutable_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path = excutable_path)

# 사이트의 html 구조에 기반하여 크롤링을 수행
source_url= "https://namu.wiki/RecentChanges" # 크롱링할 사이트 주소 정의
driver.get(source_url)
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,"app")))

# case 2
#import time
#time.sleep(10)

req = driver.page_source
soup = BeautifulSoup(req, "html.parser") # BeautifulSoup의 soup 객체로 변환

# find 사용
#contents_table = soup.find(name = "table")
#table_body = contents_table.find(name="tbody")
#table_rows = table_body.find_all(name="tr")
table_rows = soup.select("table tbody tr")

In [3]:
len(table_rows)

108

[페이지 링크주소 리스트 가져오기]

In [4]:
page_url_base = "https://namu.wiki" # 베이스 URL 
page_urls = [] # herf 속성값을 담기 위한 빈 리스트 생성


for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all("td")[0]
    td_url = first_td.find_all("a")
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get("href")
        if "png" not in page_url:
            page_urls.append(page_url)

In [5]:
len(page_urls)

99

In [6]:
page_urls

['https://namu.wiki/w/9%EC%9B%94%2025%EC%9D%BC',
 'https://namu.wiki/w/%EA%B8%89%EC%8B%9D%EC%99%95%20%26%20%EA%B8%89%EC%8B%9D%EA%B1%B8%EC%A6%88/%EB%93%B1%EC%9E%A5%EC%9D%B8%EB%AC%BC/%EA%B8%89%EC%8B%9D%EC%99%95%20%ED%95%99%EA%B5%90',
 'https://namu.wiki/w/%ED%8B%80:%EB%AF%B8%EA%B5%AD%EC%9D%98%20%EA%B3%A0%EB%93%B1%20%EA%B5%B0%EC%82%AC%EB%8C%80%ED%95%99',
 'https://namu.wiki/w/NT%EB%85%B8%EB%B2%A8',
 'https://namu.wiki/w/%EB%B2%84%EC%B8%84%EC%96%BC%20%EC%9C%A0%ED%8A%9C%EB%B2%84/%EB%AA%A9%EB%A1%9D/%ED%95%9C%EA%B5%AD',
 'https://namu.wiki/w/%EC%9D%B4%EB%B0%A9%EC%9B%90(%ED%83%9C%EC%A2%85%20%EC%9D%B4%EB%B0%A9%EC%9B%90)',
 'https://namu.wiki/w/%EC%8B%B8%EC%9D%B4(%ED%95%9C%EB%B0%98%EB%8F%84%EC%9D%98%20%EA%B3%B5%EB%A3%A1%20%EC%8B%9C%EB%A6%AC%EC%A6%88)',
 'https://namu.wiki/w/%EB%AA%A8%EC%95%BC(%EC%9D%B8%ED%84%B0%EB%84%B7%20%EB%B0%A9%EC%86%A1%EC%9D%B8)',
 'https://namu.wiki/w/%EB%AC%BC%EC%A0%81%EB%B6%84%ED%95%A0',
 'https://namu.wiki/w/%ED%95%98%EC%BD%94%EC%A6%88%EB%A9%94%20~%ED%8C%8C%EC%B6%9C%EC%